# Lambda : 2 : Conversational Session without History and SOP

In [19]:
import boto3
import json
import os
import sys
import base64
import time
import tzlocal
import re
import pandas as pd
from botocore.exceptions import ClientError

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

##Put data here
##Loading JSON to read data
file_path = "Conversation_2_14_formatted.json"
# Open the JSON file for reading
with open(file_path, 'r') as file:
    # Parse the JSON file
    data = json.load(file)

##Processing data to fetch role and content
def data_preprocessing(transcription):
    convo = ""
    for i in range(len(transcription['transcriptions'])):
        convo = convo + transcription['transcriptions'][i]['ParticipantRole'] + ": " + transcription['transcriptions'][i]['Content']
        convo += "\n"
    return convo

def data_postprocessing(data):
    result = ""
    start_index = data.find("{")
    end_char_indices = [i.start() for i in re.finditer("}",data)]
    end_index = end_char_indices[len(end_char_indices)-1]
    result = data[start_index:end_index+1]
    
    return result

def get_data_in_text(data):
    text_data = ""
    keys = list(data.keys())
    for key in keys:
        value = data[key]
        text_data += f"The {key} is {value}"
    return text_data

final_transcript = data_preprocessing(data)
##Change/Update the preprocessing logic

#Defining function to connect to Bedrock LLM
def load_claude2(bedrock_runtime , prompt , temp , top_p,top_k):
    try:
        body = {
            "prompt": prompt,
            "temperature": temp,
            "top_p": top_p,
            "top_k":top_k,
            "max_tokens_to_sample": 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="anthropic.claude-v2", body=json.dumps(body), accept="application/json", contentType="application/json"
                 )
        
        response_body = json.loads(response["body"].read())
        completion = response_body.get("completion")

        return completion

    except ClientError:
        logging.error("Couldn't invoke Claude 2")
        raise

#Defining function to summarize context
def load_text_summariser_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt" : prompt,
            "temperature" : temp,
            "top_p" : top_p,
            "max_gen_len" : 200
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logger.error("Couldn't invoke Llama 2")
        raise
        
#Defining function to summarize context
def load_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt" : prompt,
            "temperature" : temp,
            "top_p" : top_p,
            "max_gen_len" : 1000
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logger.error("Couldn't invoke Llama 2")
        raise
        
#Defining LLM function for the prompt generator for entity extraction  
def enrollment_prompt_generator(conversation,entities):
    prompt_claude = """Human: \"""" + conversation + """\"

    The above is a transcript between a call center agent and an insurance subscriber or patient. Identify and extract key entities such as \"""" + entities + """\" from the transcript. Include only the information present.

    Output the results as a structured JSON containing only the extracted fields.

    Assistant:
    """

    return prompt_claude

def summarisation_prompt_generator(context):
    prompt_llama = f"""
Instruction: "Summarise this call transcript between a patient and an agent and provide it in a precise paragraph : " :

{context}.
             
Response :  
    """
    return prompt_llama

def QnA_prompt_generator(final_summary,patient_data,question):
    prompt_llama = f"""Using the information from the conversation summary : {final_summary} 
and 
The patient enrollment data : 
{patient_data},
Answer the following Question : 
{question}

If you do not know the answer and if the Current conversation summary or the Patient enrollment data doesn't contain the answer,then 
truthfully say I don't know.
Response:
"""
    return prompt_llama

def QnA_prompt_generator_claude2(final_summary,patient_data,question):
    prompt_llama = f"""Human: \"""" + question + """\"

The above is a question/query asked by a patient to a call center agent.Using the conversation_summary which is \"""" + final_summary + """\" 
and the patient's enrollment data which is \"""" + patient_data + """\",answer the patient's question or query.

Assistant:
"""
    return prompt_llama

# Lambda handler
def get_enrollment_data(transcript):
    entities = "name of patient, status of insurance, insurance number, demographic details"
    prompt_enrollment = enrollment_prompt_generator(transcript,entities)
    enrollment_data = load_claude2(bedrock_runtime,prompt_enrollment,0,0.9,1)
    json_data = data_postprocessing(enrollment_data)
    enrollment_json_object = json.loads(json_data)
    return enrollment_json_object

def get_answer(data,question):
    transcript = data_preprocessing(data)
    patient_data = get_data_in_text(get_enrollment_data(transcript))
    summary_prompt = summarisation_prompt_generator(transcript)
    final_summary = load_text_summariser_llama2(bedrock_runtime,summary_prompt,0.5,0.9)
    QnA_prompt = QnA_prompt_generator_claude2(final_summary,patient_data,question)
    final_answer_llama_2 = load_claude2(bedrock_runtime,QnA_prompt,0.5,0.9,1)
    return final_answer_llama_2


# Lambda handler to intgerate with AWS
def lambda_handler(data,question_data):
    question = data_preprocessing(question_data)
    answer = get_answer(data,question)
    #enrollment_json_object = json.loads(answer)
    return answer#{"statusCode": 200,"body": json.dumps(answer)}

question_data = {'stream': 'TRANSCRIPTION',
 'contactId': 'aa621db9-934b-462c-bc7a-85f2e01c4c9f',
 'transcriptions': [{'ParticipantId': 'CUSTOMER',
   'ParticipantRole': 'CUSTOMER',
   'Content': "What is the customer's name?",
   'BeginOffsetMillis': 1257,
   'EndOffsetMillis': 9697,
   'Id': 'fb29489e-d06b-48b1-8e0b-519f17c4a68e',
   'Sentiment': 'NEUTRAL',
   'IssuesDetected': []}]}

#while transcripts != None:
lambda_handler(data,question_data)

" Based on the conversation summary, the customer's name is Jordan Smith."